# LLM Security - Prompt Injection
## Part 3 - Classification Using a Fine-tuned LLM

In this notebook, we load the raw dataset and fine-tune a pre-trained large language model to classify malicious prompts.
> **INPUT:** the raw dataset loaded from Hugging Face library. <br>
> **OUTPUT:** the performance analysis of fine-tuned LLM.


### 1. INITIALIZATION

In [14]:
# Import necessary libraries and modules
import pandas as pd

In [15]:
# Set display options
pd.set_option('display.max_columns', None)

### 2. LOADING DATASET

Since we are going to use a pre-trained LLM and fine-tune it using the training dataset, we need to load both training and testing data sets.

In [16]:
# Initialize data set location and file name
data_file_path = "../data/raw/"
data_file_name_train = "train-00000-of-00001-9564e8b05b4757ab"
data_file_name_test = "test-00000-of-00001-701d16158af87368"
data_file_ext = ".parquet"

# Loading data set into a pandas DataFrame
data_train = pd.read_parquet(data_file_path + data_file_name_train + data_file_ext)
data_test = pd.read_parquet(data_file_path + data_file_name_test + data_file_ext)

In [17]:
# Rename "text" column into "prompt"
data_train.rename(columns={"text":"prompt"}, inplace=True)
data_test.rename(columns={"text":"prompt"}, inplace=True)

We already explored the dataset in the previous notebooks, so we will directly proceed to the fine-tuning phase.

### 3. MODEL FINE-TUNING

In this experiment, we aim at using a pre-trained LLM and fine-tuning it on the classification task.

In the previous experiment, we used the pre-trained XLM-RoBERTa, the multilingual version of RoBERTa (Robustly optimized BERT approach), the enhanced version of BERT (Bidirectional Encoder Representations from Transformers).

To explore the effect of model fine-tuning, we fine-tune XLM-RoBERTa on our training dataset and then re-evaluate its performance in correctly predicting prompt injections.

In [18]:
# Import the model, its tokenizer, and torch library
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
import torch

In [19]:
# Load the model tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

In [20]:
# A utility function to take a batch of data and tokenize the prompts
def tokenize_batch(batch):
    return tokenizer(batch['prompt'], padding=True, truncation=True)

In [21]:
# Tokenize prompts in both training and testing datasets
prompts_train_tokenized = tokenize_batch(data_train.to_dict(orient='list'))
prompts_test_tokenized = tokenize_batch(data_test.to_dict(orient='list'))

In [22]:
# Define a dataset Class to work with PyTorch's DataLoader
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [23]:
train_dataset = CustomDataset(prompts_train_tokenized, data_train['label'])
test_dataset = CustomDataset(prompts_test_tokenized, data_test['label'])

In [79]:
# Load pre-trained Model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-large', num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


When loading the model, we get this message indicating that some weights are not initialized. This is normal since the model is pre-trained but not fine-tuned for the downstream task.

In [24]:
# Import TrainingArguments to handle the various training configurations
from transformers import TrainingArguments

# Define training arguments for fine-tuning
training_args = TrainingArguments(
    output_dir="../output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    logging_dir="../output/logs",
)

In [25]:
# Initialize a DataFrame to track the model performance
results_df = pd.DataFrame(columns=["epoch","accuracy","precision","recall","f1"])

In [28]:
# Import accuracy metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# A utility function for model evaluation during fine-tuning
def evaluate_model(trainer, epoch):
    
    # Extract predictions and labels
    predictions, labels = trainer.predictions.argmax(axis=1), trainer.label_ids
    
    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)
    
    # Calculate precision, recall, and f1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted", zero_division=1)
    
    # Append current metrics to results
    global results_df
    results_df.loc[len(results_df)] = [epoch, accuracy, precision, recall, f1]
        
    # Return
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [29]:
# Import the Trainer class
from transformers import Trainer

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: evaluate_model(p, trainer.state.epoch),
)

In [84]:
# Fine-tune the model
trainer.train()

  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.24107423424720764, 'eval_accuracy': 0.9568965517241379, 'eval_precision': 0.9580506242835455, 'eval_recall': 0.9568965517241379, 'eval_f1': 0.9568290368129876, 'eval_runtime': 41.1082, 'eval_samples_per_second': 2.822, 'eval_steps_per_second': 0.365, 'epoch': 1.0}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.1510998159646988, 'eval_accuracy': 0.9655172413793104, 'eval_precision': 0.9661117717003568, 'eval_recall': 0.9655172413793104, 'eval_f1': 0.9655274949501164, 'eval_runtime': 51.8762, 'eval_samples_per_second': 2.236, 'eval_steps_per_second': 0.289, 'epoch': 2.0}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.21839313209056854, 'eval_accuracy': 0.9741379310344828, 'eval_precision': 0.9754529514903565, 'eval_recall': 0.9741379310344828, 'eval_f1': 0.9741436973820781, 'eval_runtime': 51.0003, 'eval_samples_per_second': 2.274, 'eval_steps_per_second': 0.294, 'epoch': 3.0}
{'train_runtime': 9351.2239, 'train_samples_per_second': 0.175, 'train_steps_per_second': 0.022, 'train_loss': 0.29199080536330957, 'epoch': 3.0}


TrainOutput(global_step=207, training_loss=0.29199080536330957, metrics={'train_runtime': 9351.2239, 'train_samples_per_second': 0.175, 'train_steps_per_second': 0.022, 'train_loss': 0.29199080536330957, 'epoch': 3.0})

In [85]:
# Print detailed results
results_df

,epoch,accuracy,precision,recall,f1
0,1.0,0.956897,0.958051,0.956897,0.956829
1,2.0,0.965517,0.966112,0.965517,0.965527
2,3.0,0.974138,0.975453,0.974138,0.974144


In [86]:
# Evaluate the final model
final_results = trainer.evaluate()
print(final_results)

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.21839313209056854, 'eval_accuracy': 0.9741379310344828, 'eval_precision': 0.9754529514903565, 'eval_recall': 0.9741379310344828, 'eval_f1': 0.9741436973820781, 'eval_runtime': 46.7168, 'eval_samples_per_second': 2.483, 'eval_steps_per_second': 0.321, 'epoch': 3.0}


The fine-tuning is finished now, we will save the model to a local path.

This step is preferred so we can reload it later for evaluation and inference without the need to repeat the fine-tuning process.

In [30]:
# Set local model path
models_path = "../models/xlm_roberta"

In [88]:
# Save the model and tokenizer for a later use 
model.save_pretrained(models_path)
tokenizer.save_pretrained(models_path)

('../models/xlm_roberta\\tokenizer_config.json',
 '../models/xlm_roberta\\special_tokens_map.json',
 '../models/xlm_roberta\\sentencepiece.bpe.model',
 '../models/xlm_roberta\\added_tokens.json')

### 4. RESULT ANALYSIS

In [31]:
# Load the locally-stored fine-tuned model
model = XLMRobertaForSequenceClassification.from_pretrained(models_path)

In [32]:
# Evaluate the final model
final_results = trainer.evaluate()
print(final_results)

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.21839313209056854, 'eval_accuracy': 0.9741379310344828, 'eval_precision': 0.9754529514903565, 'eval_recall': 0.9741379310344828, 'eval_f1': 0.9741436973820781, 'eval_runtime': 7.1085, 'eval_samples_per_second': 16.318, 'eval_steps_per_second': 2.11}


In [39]:
# Print detailed results
results_df.drop("epoch", axis="columns")

,accuracy,precision,recall,f1
0,0.974138,0.975453,0.974138,0.974144


The final result demonstrate the following:
- The model achieved an impressive accuracy, which suggests that the model is effective in making correct predictions for the prompt injection classification problem.
- The precision indicates that when the model predicts a positive class, it is correct most of the time.
- The high recall also indicates that the model is able to capture the majority of the actual positive instances.
- The F1 score is high and suggests that the model is performing well in both precision and recall.

Overall, the results indicate that the fine-tuned RoBERTa model is highly effective for the classification task, achieving a high level of accuracy, precision, recall, and F1 score. A next step might be to further analyze the wrong predictions to make a sense of any potential patterns the model might have missed or was unable to capture in the dataset.
